In [ ]:
import pandas as pd
import sqlalchemy
import os
from tqdm import tqdm
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Conectar a MySQL utilizando las credenciales del archivo .env
engine = sqlalchemy.create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Ruta de la carpeta con los archivos
folder_path = '/Dataset'

# Columnas a mantener
columns_to_keep = ['Pickup_datetime', 'DropOff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles', 'trip_time', 'driver_pay']

# Columnas a eliminar
columns_to_drop = ['Hvfhs_license_num', 'Dispatching_base_num', 'originating_base_num', 'request_datetime', 'on_scene_datetime', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag', 'base_passenger_fare', 'tolls', 'tips']

# Leer todos los archivos en la carpeta y estandarizar las columnas
for filename in os.listdir(folder_path):
    if filename.endswith('.parquet'):
        file_path = os.path.join(folder_path, filename)
        
        print(f"¿Quieres procesar el archivo {filename}? (sí/no)")
        answer = input().strip().lower()
        
        if answer == 'sí':
            try:
                # Leer archivo .parquet
                df = pd.read_parquet(file_path)
                
                # Eliminar columnas innecesarias
                df = df.drop(columns=columns_to_drop, errors='ignore')
                
                # Asegurarse de que solo se mantengan las columnas deseadas
                df = df[columns_to_keep]
                
                # Dividir el DataFrame en bloques de 10,000 filas
                num_chunks = len(df) // 10000 + 1
                
                for i in tqdm(range(num_chunks), desc=f"Procesando {filename}", unit="chunk"):
                    chunk = df[i * 10000:(i + 1) * 10000]
                    chunk.to_sql(name='taxi_fhv_data', con=engine, if_exists='append', index=False)
                
                print(f"Datos del archivo {filename} cargados exitosamente.")
            
            except FileNotFoundError:
                print(f"Archivo {file_path} no encontrado. Saltando.")
            except KeyError as e:
                print(f"Error de columna en {file_path}: {e}. Saltando.")
        else:
            print(f"Omitiendo el archivo {filename}.")

print("Proceso de estandarización y carga completo.")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '/path/to/folder'